In [9]:
import pandas as pd
from src import visualization, utils, impute

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')


loading the test and train datasets.

In [11]:
encoded_df = train_df.apply(utils.Utils.map_apply, axis='columns').drop(columns=['Ticket','PassengerId'])
full_dummy_df = pd.get_dummies(encoded_df,columns=['Pclass','Sex','Name','Embarked','Cabin'])
cabin_dummy = pd.get_dummies(encoded_df, columns=['Pclass','Sex','Name','Embarked'])
print(encoded_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB
None


Encoding/one-hot encoding the categorical attributes of the full training dataset.
Then doing the same but without one-hot encoding cabin as we're preparing to first predict the cabin level of the missing values(687 NaN values).

In [17]:
knn_model_cabin = impute.Impute.knn(cabin_dummy, 4, ('Pclass_1','Pclass_2','Pclass_3'), 'Cabin')[0]
x_pred_cabin = impute.Impute.x_to_pred(cabin_dummy, ('Pclass_1','Pclass_2','Pclass_3'), 'Cabin')
knn_cabin_pred = knn_model_cabin.predict(x_pred_cabin)

print('\n There are ',len(knn_cabin_pred), ' estimated values')

 

##### knn on  Cabin  #####
   

##### vecor features #####
  ('Pclass_1', 'Pclass_2', 'Pclass_3')  

##### SCORE #####
  0.3480392156862745

 There are  687  estimated values


We predict the 687 missing values by running a knn model with the passenger class attributes.

In [34]:
encoded_df = train_df.apply(utils.Utils.map_apply, axis='columns').drop(columns=['Ticket','PassengerId'])
full_dummy_df = pd.get_dummies(encoded_df,columns=['Pclass','Sex','Name','Embarked','Cabin'])

cabin_dummy = pd.get_dummies(encoded_df, columns=['Pclass','Sex','Name','Embarked'])

knn_model_cabin = impute.Impute.knn(cabin_dummy, 4, ('Pclass_1','Pclass_2','Pclass_3'), 'Cabin')[0]
x_pred_cabin = impute.Impute.x_to_pred(cabin_dummy, ('Pclass_1','Pclass_2','Pclass_3'), 'Cabin')
knn_cabin_pred = knn_model_cabin.predict(x_pred_cabin)

cab_impute_df = impute.Impute.replace(encoded_df, knn_cabin_pred, 'Cabin')
cab_impute_enc = cab_impute_df.apply(utils.Utils.map_apply, axis='columns')

print(cab_impute_enc.info())

 

##### knn on  Cabin  #####
   

##### vecor features #####
  ('Pclass_1', 'Pclass_2', 'Pclass_3')  

##### SCORE #####
  0.3480392156862745
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Cabin       891 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB
None


We have successfully impute the NaN cabin values with the estimations.

In [35]:
full_dummy_df = pd.get_dummies(cab_impute_enc, columns=['Pclass','Sex','Name','Embarked','Cabin'])

reg_model_age = impute.Impute.reg(full_dummy_df, 5, ('Name_mr', 'Name_mrs', 'Name_miss','Parch','SibSp','Pclass_1','Pclass_3','Cabin_A','Cabin_C','Cabin_D','Cabin_E'), 'Age')[0]
x_pred_age = impute.Impute.x_to_pred(full_dummy_df, ('Name_mr', 'Name_mrs', 'Name_miss','Parch','SibSp','Pclass_1','Pclass_3','Cabin_A','Cabin_C','Cabin_D','Cabin_E'), 'Age')
reg_age_pred = reg_model_age.predict(x_pred_age)

age_impute_df = impute.Impute.replace(cab_impute_df, reg_age_pred, 'Age')
age_impute_enc = age_impute_df.apply(utils.Utils.map_apply, axis='columns')

# impute embarked
embarked_dummy = pd.get_dummies(age_impute_enc, columns=['Pclass','Sex','Name','Cabin'])

knn_model_embarked = impute.Impute.knn(embarked_dummy, 4, ('Name_mr', 'Name_miss','Pclass_1','Pclass_2','Pclass_3','Cabin_B','Cabin_C','Cabin_D'), 'Embarked')[0]
x_pred_embarked = impute.Impute.x_to_pred(embarked_dummy, ('Name_mr', 'Name_miss','Pclass_1','Pclass_2','Pclass_3','Cabin_B','Cabin_C','Cabin_D'), 'Embarked')
knn_embarked_pred = knn_model_embarked.predict(x_pred_embarked)

emb_impute_df = impute.Impute.replace(age_impute_df, knn_embarked_pred, 'Embarked')
emb_impute_enc = emb_impute_df.apply(utils.Utils.map_apply, axis='columns')

train_imputed_dummy = pd.get_dummies(emb_impute_enc, columns=['Pclass','Sex','Name','Embarked','Cabin'])

print(emb_impute_enc.info())

 

##### Lin. Reg. on  Age  #####
   

##### COEFFICIENTS #####
  [ 7.87898514 10.3902856  -2.42587744 -1.25716393 -3.30674707  6.68291961
 -4.06806004  8.71707376  1.88203242  3.90980924  2.91791328]  

##### IN RSCORE #####
  0.35078705010509137  

##### OUT RSCORE #####
  0.307561656740402
 

##### knn on  Embarked  #####
   

##### vecor features #####
  ('Name_mr', 'Name_miss', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Cabin_B', 'Cabin_C', 'Cabin_D')  

##### SCORE #####
  0.6985376827896513
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Cabin       891 non-null object
Embarked    891 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB
None


By applying the same principal to the other attributes that are missing values, we end up with a fully imputed training dataset. Note that for the continous values, we used a regression. 

In [36]:
enc_test_df = test_df.apply(utils.Utils.map_apply, axis='columns').drop(columns=['Ticket','PassengerId'])
test_dummy_df = pd.get_dummies(enc_test_df,columns=['Pclass'])
print(enc_test_df.info())
#impute cabin

x_pred_cabin_test = impute.Impute.x_to_pred(test_dummy_df, ('Pclass_1','Pclass_2','Pclass_3'), 'Cabin')
knn_cab_pred_test = knn_model_cabin.predict(x_pred_cabin_test)

cab_impute_test_df = impute.Impute.replace(enc_test_df, knn_cab_pred_test, 'Cabin')

cab_impute_test_enc = cab_impute_test_df.apply(utils.Utils.map_apply, axis='columns')
cab_impute_test_d = pd.get_dummies(cab_impute_test_enc, columns=['Pclass','Sex','Name','Embarked','Cabin'])

# impute age

x_pred_age_test = impute.Impute.x_to_pred(cab_impute_test_d, ('Name_mr', 'Name_mrs', 'Name_miss','Parch','SibSp','Pclass_1','Pclass_3','Cabin_A','Cabin_C','Cabin_D','Cabin_E'), 'Age')
reg_age_pred_test = reg_model_age.predict(x_pred_age_test)

age_impute_test_df = impute.Impute.replace(cab_impute_test_enc, reg_age_pred_test, 'Age')
age_impute_test_enc = age_impute_test_df.apply(utils.Utils.map_apply, axis='columns')
age_impute_test_d = pd.get_dummies(age_impute_test_enc, columns=['Pclass','Sex','Name','Embarked','Cabin'])

# impute fare using train data for the estimation

reg_model_fare = impute.Impute.reg(train_imputed_dummy, 5, ('Age', 'SibSp', 'Parch', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
                                                      'Name_miss', 'Name_mr', 'Name_mrs', 'Name_staff', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
                                                      'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G') , 'Fare')[0]
x_pred_fare = impute.Impute.x_to_pred(age_impute_test_d, ('Age', 'SibSp', 'Parch', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
                                                      'Name_miss', 'Name_mr', 'Name_mrs', 'Name_staff', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
                                                      'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G'), 'Fare')
reg_fare_pred = reg_model_fare.predict(x_pred_fare)

fare_impute_df = impute.Impute.replace(age_impute_test_enc, reg_fare_pred, 'Fare')
fare_impute_enc = fare_impute_df.apply(utils.Utils.map_apply, axis='columns')
print(fare_impute_enc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
Pclass      418 non-null int64
Name        418 non-null object
Sex         418 non-null object
Age         332 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        417 non-null float64
Cabin       91 non-null object
Embarked    418 non-null object
dtypes: float64(2), int64(3), object(4)
memory usage: 29.5+ KB
None
 

##### Lin. Reg. on  Fare  #####
   

##### COEFFICIENTS #####
  [ -0.1993639    5.96135074  10.29931478  33.6756009  -14.11569503
 -19.55990587   2.47334176  -2.47334176   4.54783743   4.56596803
  -2.07449568  -7.03930979   7.20742249  -2.70313742  -4.50428507
   0.83410617  52.7394313   45.73630044  20.29734395  11.93133758
  23.7039143   17.33781442]  

##### IN RSCORE #####
  0.4828960483351825  

##### OUT RSCORE #####
  0.4594993670233196
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (tota

For the NaN values in the test set, we did the same as for the training set. Note that the models we used for the test set were trained on the training set as it contains more observations, therefore better predictive power.

In [42]:
from sklearn.preprocessing import StandardScaler
test_pp = pd.get_dummies(fare_impute_enc,columns=['Pclass','Sex','Name','Embarked','Cabin'])
train_pp = pd.get_dummies(emb_impute_enc,columns=['Pclass','Sex','Name','Embarked','Cabin'])


test_pp.loc[:,('Age','Fare')] = StandardScaler().fit_transform(test_pp.loc[:,('Age','Fare')])
train_pp.loc[:,('Age','Fare')] = StandardScaler().fit_transform(train_pp.loc[:,('Age','Fare')])

test_pp.to_pickle("./pp_test.pkl")
test_pp.to_csv("./pp_test.csv")

train_pp.to_pickle("./pp_train.pkl")
train_pp.to_csv("./pp_train.csv")

The last step is to one-hot encode and scale the continuous attributes in both the test and train dataset.